In [21]:
import numpy as np
import pandas as pd
from math import floor
from scipy.stats import norm

In [22]:
S_0 = 36
K = 40
T = 0.75
Sigma = 0.32
r = 0.04
q = 0.015

tau_final = T * (Sigma ** 2) / 2
x_compute = np.log(S_0 / K)
x_left = x_compute + (r - q - Sigma ** 2 / 2) * T - 3 * Sigma * np.sqrt(T)
x_right = x_compute + (r - q - Sigma ** 2 / 2) * T + 3 * Sigma * np.sqrt(T)

a = (r - q) / Sigma ** 2 - 1 / 2
b = (a + 1) ** 2 + 2 * q / Sigma ** 2

def f(x):
    return K * np.exp(a * x) * max(1 - np.exp(x), 0)

def g_left(tau):
    return K * np.exp(a * x_left + b * tau) * (np.exp(- 2 * r * tau / Sigma ** 2) - np.exp(x_left - 2 * q * tau / Sigma ** 2))

def g_right(tau):
    return 0

In [59]:
def grid_solver(M):
    alpha = 0.45
    delta_T = tau_final / M
    N = floor((x_right - x_left) / np.sqrt(delta_T / alpha))
    delta_x = (x_right - x_left) / N
    alpha = delta_T / delta_x ** 2
    grid = np.zeros((M+1, N+1), dtype=float)
    for i in range(M + 1):
        t_i = delta_T * i
        grid[i, 0] = g_left(t_i)
        grid[i, N] = g_right(t_i)

    for i in range(N + 1):
        x_i = x_left + i * delta_x
        grid[0, i] = f(x_i)

    for i in range(1, M + 1):
        for j in range(1, N):
            grid[i, j] = alpha * grid[i-1, j-1] + (1 - 2*alpha) * grid[i-1, j] + alpha * grid[i-1, j+1]
    
    return grid

def bsm_solver(S):
    d1 = (np.log(S / K) + (r - q + 0.5 * Sigma**2) * T) / (Sigma * np.sqrt(T))
    d2 = d1 - Sigma * np.sqrt(T)

    return K * np.exp(-r * T) * norm.cdf(-d2) - S * np.exp(-q * T) * norm.cdf(-d1)

output1 = np.round(grid_solver(4), 6)
pd.DataFrame(output1).to_csv('./grid4.csv')

In [60]:
grid_solver(4)

array([[31.45714068, 27.18156861, 22.70125296, 17.98027406, 12.97887372,
         7.65298169,  1.95368624,  0.        ,  0.        ,  0.        ,
         0.        ,  0.        ],
       [31.38461484, 27.09554787, 22.60014097, 17.86245811, 12.84254214,
         7.49610039,  3.52736233,  0.82081957,  0.        ,  0.        ,
         0.        ,  0.        ],
       [31.31155144, 27.00885644, 22.49835142, 17.743881  , 12.70535596,
         8.07492729,  4.05765966,  1.61308522,  0.34485822,  0.        ,
         0.        ,  0.        ],
       [31.2379475 , 26.92154736, 22.39585207, 17.62453861, 12.87681312,
         8.33254009,  4.71840923,  2.10731453,  0.73280135,  0.14488835,
         0.        ,  0.        ],
       [31.16380006, 26.83361433, 22.29265833, 17.63444883, 12.96229141,
         8.72332898,  5.13982373,  2.62685051,  1.06328267,  0.33102023,
         0.06087323,  0.        ]])

In [31]:
M_list = [4, 16, 64, 256]
col = ['e', 'ep', 'e2', 'ep2', 'er', 'erp', 'd', 'g', 't']
df = pd.DataFrame(index=M_list, columns=col)
V_exact = bsm_solver(S_0)

,e,ep,e2,ep2,er,erp,d,g,t
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
16,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
64,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
256,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [61]:
e = []
e2 = []
er = []
d = []
g = []
t = []

for M in M_list:
    delta_T = tau_final / M
    N = floor((x_right - x_left) / np.sqrt(delta_T / alpha))
    i = floor((x_compute - x_left) / np.sqrt(delta_T / alpha))
    grid = grid_solver(M)
    x = np.linspace(x_left, x_right, N+1)
    S = K * np.exp(x)
    u = grid[-1]

    V = np.exp(- a * x - b * tau_final) * u
    V_appro = ((S[i+1]-S_0) * V[i] + (S_0-S[i]) * V[i+1]) / (S[i+1] - S[i])
    V_error = abs(V_appro - V_exact)

    u_appro = ((x[i+1]-x_compute) * u[i] + (x_compute-x[i]) * u[i+1]) / (x[i+1] - x[i])
    V_appro2 = np.exp(-a*x_compute-b*tau_final) * u_appro
    V_error2 = abs(V_appro2 - V_exact)
    
    e.append(V_error)
    e2.append(V_error2)

    N_rms = 0
    v_rms = 0
    for k in range(N+1):
        flag = (V[k] > 0.00001 * S_0)
        if flag:
            N_rms += 1
            v_appro = np.exp(-a*x[k]-b*tau_final) * u[k]
            v_exact = bsm_solver(S[k])
            v_rms += ((v_appro - v_exact) / v_exact) ** 2
    e_rms = np.sqrt(v_rms / N_rms)
    er.append(e_rms)

    delta = (V[i+1]-V[i]) / (S[i+1]-S[i])
    gamma = ((V[i+2]-V[i+1])/(S[i+2]-S[i+1]) - (V[i]-V[i-1])/(S[i]-S[i-1])) / ((S[i+2]+S[i+1])/2 - (S[i]+S[i-1])/2)
    
    u2 = grid[-2]
    V2 = np.exp(- a * x - b * (tau_final - delta_T)) * u2
    V_appro_dt = ((S[i+1]-S_0) * V2[i] + (S_0-S[i]) * V2[i+1]) / (S[i+1] - S[i])
    theta = (V_appro_dt - V_appro) / delta_T

    d.append(delta)
    g.append(gamma)
    t.append(theta)

df['e'] = e
df['ep'] = df['e'] / df['e'].shift(1)
df['e2'] = e2
df['ep2'] = df['e2'] / df['e2'].shift(1)
df['er'] = er
df['erp'] = df['er'] / df['er'].shift(1)
df['d'] = d
df['g'] = g
df['t'] = t

In [63]:
output2 = np.round(df, 6)
output2.to_csv('./grid.csv')